In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import ChatOllama
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings, HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [2]:

llm = ChatOllama(model="mistral:instruct", temperature=0)
# For retrieval
# embeddings = OllamaEmbeddings(model="nomic-embed-text",show_progress=True,temperature=0.5, query_instruction="search_query:", embed_instruction="search_document:")

# For creating vector db or fast retrieval
embeddings = HuggingFaceBgeEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code':True,
    },
    encode_kwargs={'normalize_embeddings': True},
    query_instruction = "search_query:",
    embed_instruction = "search_document:"
)


c:\Users\ADMIN\.conda\envs\ml-env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
<All keys matched successfully>


# Initialize db

In [2]:
loader = DirectoryLoader(path="Data", glob="*.txt",use_multithreading=True)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=200, add_start_index=True)
docs_chunks = text_splitter.split_documents(docs)
db = Chroma.from_documents(docs_chunks, embeddings, persist_directory="./sentence_chroma_db")


KeyboardInterrupt: 

# Inference

In [3]:
# load doc
db = Chroma(persist_directory="./sentence_chroma_db", embedding_function=embeddings, collection_metadata={"hnsw:space": "cosine"})

In [4]:
# test vector db
query = "Tell me about Elden Ring"
docs = db.similarity_search(query, k=2)

# print results
print(docs[0].page_content)

Shinkansen

link: https://simple.wikipedia.org/wiki/Shinkansen

The Shinkansen (Japanese: 新幹線, pronounced [ɕiŋkaꜜɰ̃seɴ], lit. ''new main line'') is a group of high-speed rail lines in Japan, upon which the famous "Bullet Trains" run. It is the world's first high-speed rail. The Shinkansen are run by the many JR companies. In the past, Japan Railway was called Japanese National Railways. It is now one group of private companies. JR also runs commuter trains and inter-city trains around Tokyo. The name "Bullet Train" is the English translation of the Japanese word dangan ressha (弾丸列車), which was the name given to the project while it was being studied in the 1940s. Nowadays, the trains are called Shinkansen trains. The name Shinkansen means "New Trunk Line". The trains are called "Super Expresses". Japan was the first country to build railway lines for high-speed travel. Because Japan has many mountains, the network that already existed was made of 3'6" gauge (1067 mm) narrow gauge lines

In [5]:
# prompt for model
if llm.model.find("mistral") >= 0:
    prompt  = PromptTemplate.from_template(
"""[INST]
You are an assistant. You will be provided the context relevant to the user's question and output the answer to the user.
If the retrieved context was not relevant at all to the question, answer the question with your knowledge, say sorry to the user when you can not find the answer.
You can also add some information if you know something about it, as long as your knowledge is relevant to the context. 
context: {context}
question: {question}[/INST]
"""
    )
    qa_chain = prompt | llm
elif llm.model.find("phi3") >= 0:
    prompt  = PromptTemplate.from_template(
"""<|user|>
You are an assistant. You will be provided the context relevant to the user's question and output the answer to the user.
If the retrieved context was not relevant at all to the question, answer the question with your knowledge, say sorry to the user when you can not find the answer.
You can also add some information if you know something about it, as long as your knowledge is relevant to the context. 
context: {context}
question: {question} <|end|> <|assistant|>
"""
    )
    qa_chain = prompt | llm


In [6]:
# test spAcy NER 
import spacy
nlp = spacy.load("en_core_web_trf")
query = "Tell me about that new game Elden Ring by From Software"
nlp_object = nlp(query)
print([(y.text, y.label_) for y in nlp_object.ents])

[('Elden Ring', 'WORK_OF_ART'), ('From Software', 'ORG')]


In [7]:
def where_document_query(nlp_object_ents: list):
    if len(nlp_object_ents) == 1:
        return  {"$contains": nlp_object_ents[0].text}
    if len(nlp_object_ents) > 1: 
        return {"$or": [{"$contains": ent.text} for ent in nlp_object_ents]}
    return None

In [8]:
llm.model

'mistral:instruct'

In [9]:
query = "Tell me about 39th G8 summit"
nlp_object = nlp(query)
db.similarity_search(
    query,
    k=3,
    where_document=where_document_query(nlp_object.ents)
)

[Document(page_content='39th G8 summit\n\nlink: https://simple.wikipedia.org/wiki/39th_G8_summit\n\nThe 39th G8 summit took place in Northern Ireland on 17th and 18th June 2013. The meeting was at the Lough Erne Golf Resort in Enniskillen,  Northern Ireland In the Queen\'s speech at the state opening of Parliament in 2012, Elizabeth II said that "my government will use this opportunity to promote international security and prosperity. "[1] The gathering of the Group of Eight (G8) will be the 39th meeting in a series which began in 1976. [2] Previous G8 summits have been hosted by the United Kingdom at London (1977, 1984, 1991) and Birmingham (1998) in England. The 2005 summit was in Scotland at Gleneagles. [3] The G8 and the summit are part of a consultation process. The G8 is not an international organization. [4] It is an informal group. [5] The participants will be the "core members" of the group:[6] A number of national leaders are traditionally invited to attend the summit. They a

In [10]:
# main flow
import pprint

query = "Tell me about 40th G7 summit"
nlp_object = nlp(query)

pprint.pprint(
    qa_chain.invoke({
        "question": query,
        "context": "/n".join(
            result.page_content for result in db.similarity_search(
                query,
                k=3,
                where_document=where_document_query(nlp_object.ents)
            )
        ),
    }).content
)


(' The 40th G7 summit was held in Brussels, Belgium on June 4-5, 2014. '
 'Originally planned to be held at Sochi, Russia, the summit was moved due to '
 "Russia's involvement in the 2014 Crimean crisis. As a result, Russia was not "
 'invited to participate. The core members of the G7 who attended were Canada, '
 'the European Commission, France, Germany, Italy, Japan, and the United '
 'Kingdom.\n'
 '\n'
 'At the summit, the leaders of the G7 nations issued a joint statement '
 'condemning Moscow for its "continuing violation" of Ukraine\'s sovereignty '
 'and announced their preparedness to impose further sanctions on Russia over '
 'its actions in Ukraine. The summit was the first since Russia was expelled '
 'from the group following its annexation of Crimea in March.\n'
 '\n'
 'Planning for the summit had to anticipate protest groups and other '
 'activists, and some saw it as a profit-generating event, such as the G8 '
 'Summit magazines which have been published since 1998. The

In [9]:
db._collection_metadata